## Data Download and Pre Processing

- Download dataset from Kaggle 
- API code : kaggle datasets download -d shanwizard/modest-museum-dataset

In [2]:
#Insall Kaggle
#!pip install kaggle
#kaggle datasets download -d shanwizard/modest-museum-dataset

In [3]:
#!unzip -q modest-museum-dataset.zip

Directory Structure
The dataset comprises seven parts, which are stored inside the root directory.

bg: (d) Background images
fg: (d) Foreground images
fg_mask: (d) Mask of foreground images
bg_fg: (d) Images where foregrounds are overlayed on top of backgrounds
bgfgmask: (d) Mask of overlayed foreground-background images
bgfgdepth_map: (d) Depth maps of overlayed foreground-background images
file_map.txt: (f) Relations between files across different parts of the dataset
bbox.txt: (f) Bounding box of foreground in each background

### Note
- The size of dataset is 6gb(zipped). 
*As the data set is huge and not being able to process it, I have sampled a small subset from the main dataset*
- Below are the steps to select a subset of images randomly(not a great approach) 
- Divide data into train and val dataset

In [6]:
import os
import pandas as pd
import random

In [100]:
names = os.listdir('bg_fg')

In [4]:
len(names)

400000

In [6]:
train_imgs = random.sample(names, 20000)

In [10]:
val_imgs = random.sample(list(set(names)-set(train_imgs)),2000)

## Annotation file format:

Existing

ID  | xmin | ymin  | width | Height 

Required

ID  | xmin ymin xmax ymax Label




In [102]:
bbox = pd.read_csv('bbox.txt', names = ['ID','xmin','ymin','w','h'],sep='\t')

In [103]:
bbox.dtypes

ID      object
xmin     int64
ymin     int64
w        int64
h        int64
dtype: object

In [104]:
bbox_train = bbox[bbox['ID'].isin(train_imgs)]
bbox_val = bbox[bbox['ID'].isin(val_imgs)]

In [105]:
bbox_train.shape

(20000, 5)

In [106]:
bbox_train['xmax'] = bbox_train[['xmin','w']].sum(axis = 1)
bbox_train['ymax'] = bbox_train[['ymin','h']].sum(axis = 1)
bbox_val['xmax'] = bbox_val[['xmin','w']].sum(axis = 1)
bbox_val['ymax'] = bbox_val[['ymin','h']].sum(axis = 1)



/Users/jayko/Documents/monk/monkenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/jayko/Documents/monk/monkenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jayko/Documents/monk/monkenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [107]:
bbox_train.drop(['h','w'],axis=1, inplace=True)
bbox_val.drop(['h','w'],axis=1, inplace=True)

/Users/jayko/Documents/monk/monkenv/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [108]:
bbox_train = bbox_train.astype('str')
bbox_val = bbox_val.astype('str')

bbox_train['Label'] = bbox_train['xmin']+" "+bbox_train['ymin']+" "+bbox_train['xmax']+" "+bbox_train['ymax']+" "+"person"
bbox_val['Label'] = bbox_val['xmin']+" "+bbox_val['ymin']+" "+bbox_val['xmax']+" "+bbox_val['ymax']+" "+"person"

In [109]:
bbox_train.head()

,ID,xmin,ymin,xmax,ymax,Label
2,bg001_fg001_03.jpeg,15,7,50,115,15 7 50 115 person
33,bg001_fg001_34.jpeg,119,114,154,222,119 114 154 222 person
43,bg001_fg002_04.jpeg,71,75,102,183,71 75 102 183 person
103,bg001_fg003_24.jpeg,16,102,83,210,16 102 83 210 person
131,bg001_fg004_12.jpeg,0,75,70,183,0 75 70 183 person


In [110]:
bbox_train.drop(['xmin','ymin','xmax','ymax'], axis=1, inplace=True)
bbox_val.drop(['xmin','ymin','xmax','ymax'], axis=1, inplace=True)

In [111]:
bbox_train.head()

,ID,Label
2,bg001_fg001_03.jpeg,15 7 50 115 person
33,bg001_fg001_34.jpeg,119 114 154 222 person
43,bg001_fg002_04.jpeg,71 75 102 183 person
103,bg001_fg003_24.jpeg,16 102 83 210 person
131,bg001_fg004_12.jpeg,0 75 70 183 person


### Create destination dir

In [42]:
os.mkdir("images_train")
os.mkdir("images_val")
os.mkdir("annotations")

Move the selected images to their respective folders

In [71]:
import os
import shutil
src_files = train_imgs
for file_name in src_files:
    full_file_name = os.path.join("bg_fg", file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, "images_train")
src_files = val_imgs
for file_name in src_files:
    full_file_name = os.path.join("bg_fg", file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, "images_val")       


In [119]:
bbox_train.set_index('ID',inplace=True)
bbox_train.head()

,Label
ID,
bg001_fg001_03.jpeg,15 7 50 115 person
bg001_fg001_34.jpeg,119 114 154 222 person
bg001_fg002_04.jpeg,71 75 102 183 person
bg001_fg003_24.jpeg,16 102 83 210 person
bg001_fg004_12.jpeg,0 75 70 183 person


In [120]:
bbox_val.set_index('ID',inplace=True)
bbox_val.head()

,Label
ID,
bg001_fg003_21.jpeg,55 0 122 108 person
bg001_fg006_28.jpeg,133 115 198 223 person
bg001_fg015_21.jpeg,89 101 159 209 person
bg001_fg016_23.jpeg,53 23 121 131 person
bg001_fg017_09.jpeg,40 49 92 157 person


In [123]:
bbox_val.to_csv("annotations/bbox_val.csv")
bbox_train.to_csv("annotations/bbox_train.csv")

'/Users/jayko/Downloads/modest_museum_dataset'

# After sampling the data, convert it into a zip file and upload it to kaggle for using it in colab